In [1]:
import numpy as np
import cv2
import os
#creating dataset

def create_directory(directory):
    """
    Create a directory if it doesn't exist.

    Parameters:
        directory (str): The path of the directory to be created.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

# Create 'images' directory if it doesn't exist
create_directory('images')

# Load the pre-trained face cascade classifier
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open a connection to the default camera (camera index 0)
cam = cv2.VideoCapture(0)

# Set camera dimensions
cam.set(3, 640)
cam.set(4, 480)

# Initialize face capture variables
count = 0
face_id = input('\nEnter user id (MUST be an integer) and press <return> -->  ')
print("\n[INFO] Initializing face capture. Look at the camera and wait...")

while True:
    # Read a frame from the camera
    ret, img = cam.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Draw a rectangle around the detected face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Increment the count for naming the saved images
        count += 1

        # Save the captured image into the 'images' directory
        cv2.imwrite(f"./images/Users-{face_id}-{count}.jpg", gray[y:y+h, x:x+w])

        # Display the image with rectangles around faces
        cv2.imshow('image', img)

    # Press Escape to end the program
    k = cv2.waitKey(100) & 0xff
    if k < 30:
        break

    # Take 30 face samples and stop video. You may increase or decrease the number of
    # images. The more, the better while training the model.
    elif count >= 30:
        break

print("\n[INFO]Success! Exiting Program.")

# Release the camera
cam.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


[INFO] Initializing face capture. Look at the camera and wait...

[INFO]Success! Exiting Program.


In [3]:
#train
import cv2
import numpy as np
from PIL import Image
import os


# Directory path where the face images are stored.
path = './images/'
recognizer = cv2.face.LBPHFaceRecognizer_create()
print("\n[INFO] Training...")
# Haar cascade file for face detection
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def getImagesAndLabels(path):
    """
    Load face images and corresponding labels from the given directory path.

    Parameters:
        path (str): Directory path containing face images.

    Returns:
        list: List of face samples.
        list: List of corresponding labels.
    """
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []

    for imagePath in imagePaths:
        # Convert image to grayscale
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img, 'uint8')
        
        # Extract the user ID from the image file name
        id = int(os.path.split(imagePath)[-1].split("-")[1])

        # Detect faces in the grayscale image
        faces = detector.detectMultiScale(img_numpy)

        for (x, y, w, h) in faces:
            # Extract face region and append to the samples
            faceSamples.append(img_numpy[y:y+h, x:x+w])
            ids.append(id)

    return faceSamples, ids

faces, ids = getImagesAndLabels(path)
print(len(faces))
# Train the recognizer with the face samples and corresponding labels
recognizer.train(faces, np.array(ids))

# Save the trained model into the current directory
recognizer.write('trainer.yml')

print("\n[INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


[INFO] Training...
219

[INFO] 4 faces trained. Exiting Program


In [1]:
import cv2
import numpy as np
import os


# Create LBPH Face Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
# Load the trained model
recognizer.read('trainer.yml')


< cv2.face.LBPHFaceRecognizer 0000023180146230>

 [INFO] Exiting Program.


In [2]:
print(recognizer)
# Path to the Haar cascade file for face detection
face_cascade_Path = "haarcascade_frontalface_default.xml"

# Create a face cascade classifier
faceCascade = cv2.CascadeClassifier(face_cascade_Path)

# Font for displaying text on the image
font = cv2.FONT_HERSHEY_SIMPLEX

# Initialize user IDs and associated names
id = 0
# Don't forget to add names associated with user IDs
names = {9:"noman",2:"adil",6:"jahanzaib",10:"talal"}
# Video Capture from the default camera (camera index 0)
cam = cv2.VideoCapture("../../face_recognition/videos/q.mp4")
cam.set(3, 640)  # Set width
cam.set(4, 480)  # Set height

# Minimum width and height for the window size to be recognized as a face
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    # Read a frame from the camera
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    for (x, y, w, h) in faces:

        # Draw a rectangle around the detected face
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Recognize the face using the trained model
        id, confidence = recognizer.predict(gray[y:y + h, x:x + w])
        # Proba greater than 51
        if confidence > 51:
            try:
                # Recognized face
                name = names[id]
                confidence = "  {0}%".format(round(confidence))
            except IndexError as e:
                name = "Who are you?"
                confidence = "N/A"
        else:
            # Unknown face
            name = "Who are you?"
            confidence = "N/A"

        # Display the recognized name and confidence level on the image
        cv2.putText(img, name, (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, confidence, (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)

    # Display the image with rectangles around faces
    cv2.imshow('camera', img)

    # Press Escape to exit the webcam / program
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break

print("\n [INFO] Exiting Program.")
# Release the camera
cam.release()
# Close all OpenCV windows
cv2.destroyAllWindows()

< cv2.face.LBPHFaceRecognizer 0000023180146230>


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 